In [66]:
!pip install sqlalchemy psycopg2-binary bertopic sklearn nltk hdbscan sentence-transformers umap-learn sklearn
import nltk
nltk.download('punkt')
nltk.download('stopwords')



[nltk_data] Downloading package punkt to /Users/mthasin/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/mthasin/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [3]:
import os
import pandas as pd

from sqlalchemy import create_engine

df = pd.read_sql(
    "select MESSAGE, ticket_created_date from client_experience.dim_web_interactions where  is_client = 1 and interaction_seq = 1 and channel = 'web' limit 3000",
    con=create_engine(os.environ.get('CONN_PANTHEON'))
)
#ticket id 
df.head(5)

,message,ticket_created_date
0,I cannot cancel my limit order. An error has o...,2021-01-20 10:00:31
1,How long does a market buy take? I put an orde...,2021-01-20 10:05:29
2,"I placed 3 orders. I canceled the first 2, but...",2021-01-20 10:07:22
3,I seem to have lost 100$ without any reasons.....,2021-01-20 10:08:18
4,ERROR! Cannot cancel trade. I placed a market ...,2021-01-20 10:24:59


In [4]:
from sentence_transformers import SentenceTransformer
from umap.umap_ import UMAP
from hdbscan import HDBSCAN

embedding_model = SentenceTransformer('all-MiniLM-L6-v2')
umap_model = UMAP(n_neighbors=3, n_components=3, min_dist=0.05)
hdbscan_model = HDBSCAN(min_cluster_size=5, #test, min_samples=40,#test with 10 
                        gen_min_span_tree=True,
                        prediction_data=True)

In [5]:
import pandas as pd
import re
df.columns = df.columns.str.strip()
df = df.replace(r'\n',' ', regex=True) 
df = df.replace(r'\r',' ', regex=True) 
df = df.replace(r"(@\[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", "",regex=True)
df.head()

,message,ticket_created_date
0,I cannot cancel my limit order An error has oc...,2021-01-20 10:00:31
1,How long does a market buy take I put an order...,2021-01-20 10:05:29
2,I placed 3 orders I canceled the first 2 but y...,2021-01-20 10:07:22
3,I seem to have lost 100 without any reasons Th...,2021-01-20 10:08:18
4,ERROR Cannot cancel trade I placed a market bu...,2021-01-20 10:24:59


In [184]:
from bertopic import BERTopic

model = BERTopic(
    umap_model=umap_model,
    hdbscan_model=hdbscan_model,
    embedding_model=embedding_model,
    language='english',
    verbose=True,
    diversity=0.6,
    calculate_probabilities=True, #change diversity of topics 
)
topics, probs = model.fit_transform(df['message'])
model.get_topic_info()

Batches:   0%|          | 0/94 [00:00<?, ?it/s]

2022-06-23 13:36:50,262 - BERTopic - Transformed documents to Embeddings
2022-06-23 13:36:53,814 - BERTopic - Reduced dimensionality
2022-06-23 13:36:55,608 - BERTopic - Clustered reduced embeddings


,Topic,Count,Name
0,-1,620,-1_and_account_shares_have
1,0,55,0_citizenship_canadian_permanent_india
2,1,50,1_transfer_manulife_accounts_submit
3,2,49,2_mon_jai_pour_pas
4,3,40,3_split_reverse_shares_scr
...,...,...,...
164,167,6,167_nexe_dividend_buggy_row
182,181,5,181_invest_rrsp_diane_select
183,182,5,182_gaming_disappeared_scr_purchased
184,183,5,183_simpletax_nemzutkoviccom_central_labontray...


In [44]:
#testing sentiments 

from nltk.sentiment import SentimentIntensityAnalyzer

analyser = SentimentIntensityAnalyzer()

sentiment_score_list = []
sentiment_label_list = []

for i in df['message'].values.tolist():
    sentiment_score = analyser.polarity_scores(i)

    if sentiment_score['compound'] >= 0.05:
        sentiment_score_list.append(sentiment_score['compound'])
        sentiment_label_list.append('Positive')
    elif sentiment_score['compound'] > -0.05 and sentiment_score['compound'] < 0.05:
        sentiment_score_list.append(sentiment_score['compound'])
        sentiment_label_list.append('Neutral')
    elif sentiment_score['compound'] <= -0.05:
        sentiment_score_list.append(sentiment_score['compound'])
        sentiment_label_list.append('Negative')
    
df['sentiment'] = sentiment_label_list
df['sentiment score'] = sentiment_score_list
df['topic'] = topics

display(df.head(10))

,message,ticket_created_date,sentiment,sentiment score,topic
0,I cannot cancel my limit order An error has oc...,2021-01-20 10:00:31,Negative,-0.2406,151
1,How long does a market buy take I put an order...,2021-01-20 10:05:29,Neutral,0.0000,68
2,I placed 3 orders I canceled the first 2 but y...,2021-01-20 10:07:22,Negative,-0.6808,-1
3,I seem to have lost 100 without any reasons Th...,2021-01-20 10:08:18,Negative,-0.7096,-1
4,ERROR Cannot cancel trade I placed a market bu...,2021-01-20 10:24:59,Positive,0.1398,162
5,I placed 2 order for black berry at 1020 even...,2021-01-20 10:30:38,Neutral,0.0000,42
6,my account is frozen What is going on I need ...,2021-01-20 10:50:31,Neutral,0.0000,-1
7,Hello I would like to begin selfdirected tr...,2021-01-20 10:51:31,Positive,0.6124,64
8,My account is not started yet So I just want t...,2021-01-20 10:55:25,Neutral,0.0404,10
9,Id like to transfer money from my personal acc...,2021-01-20 11:00:44,Positive,0.5719,7


In [220]:
model.visualize_hierarchy()

In [246]:
import numpy as np
from scipy.cluster.hierarchy import linkage
from typing import List
from sklearn.metrics.pairwise import cosine_similarity

import plotly.graph_objects as go
import plotly.figure_factory as ff 
def hh(topic_model,
                        orientation: str = "left",
                        topics: List[int] = None,
                        top_n_topics: int = None,
                        width: int = 1000,
                        height: int = 600):

    # Select topic embeddings
    if topic_model.topic_embeddings is not None:
        embeddings = np.array(topic_model.topic_embeddings)
    else:
        embeddings = topic_model.c_tf_idf

    # Select topics based on top_n and topics args
    freq_df = topic_model.get_topic_freq()
    freq_df = freq_df.loc[freq_df.Topic != -1, :]
    if topics is not None:
        topics = list(topics)
    elif top_n_topics is not None:
        topics = sorted(freq_df.Topic.to_list()[:top_n_topics])
    else:
        topics = sorted(freq_df.Topic.to_list())

    #  embeddings
    all_topics = sorted(list(topic_model.get_topics().keys())) #gets topic number/keys
    indices = np.array([all_topics.index(topic) for topic in topics]) # for every topic 
    embeddings = embeddings[indices]

    distance_matrix = 1 - cosine_similarity(embeddings)
    
        # labels
    axis = "yaxis" if orientation == "left" else "xaxis"
    new_labels = [[[str(topics[int(x)]), None]] + topic_model.get_topic(topics[int(x)])
                  for x in fig.layout[axis]["ticktext"]]
    new_labels = ["_".join([label[0] for label in labels[:4]]) for labels in new_labels]
    new_labels = [label if len(label) < 30 else label[:27] + "..." for label in new_labels]

    return new_labels, distance_matrix

In [250]:
np.set_printoptions(threshold=np.inf)

new_labels, distance_matrix = hh(model)

linkagefun=lambda x: linkage(x, "ward")
distance_matrix = linkagefun(distance_matrix)
cutree = cluster.hierarchy.cut_tree(distance_matrix,n_clusters = 25) #also test with height parameter
cutree = cutree.tolist()

In [251]:
cutree = list(np.concatenate(cutree).flat)
new = pd.DataFrame()
new_lis  = list (zip(new_labels, cutree))
new_data = pd.DataFrame(new_lis, columns = ['topics name', 'topic'])


In [252]:
with pd.option_context('display.max_rows', None,
                       'display.max_columns', None,
                       'display.precision', 3,
                       ):
    print(new_data)

                        topics name  topic
0           79_crypto_30000_1781845      0
1              7_crypto_personal_td      1
2          30_crypto_laptop_opening      2
3          72_binance_bitcoin_theta      3
4         5_investing_would_started      4
5            90_cash_roundup_really      5
6           11_2020_slip_harvesting      0
7          98_sunlife_manulife_rrsp      5
8         85_rrsp_join_h93836049cad      6
9            163_rbc_ticker_7016637      7
10   87_fees_drastically_commiss...      8
11                 28_td_rsp_stocks      9
12              55_buy_refunds_btcc     10
13     18_furniture_platform_cancer     11
14   113_neo_questrade_certifica...     12
15           60_isa_leclair_willems      1
16    151_inquiries_appsthank_perks      1
17              37_cibc_preeti_rush      8
18            181_invest_rrsp_diane      9
19                20_excess_ken_ira      1
20          167_nexe_dividend_buggy      9
21   109_investors_215513789_qtr...     11
22      176